In [1]:
import os
import pandas as pd
import numpy as np 
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
from playlist_membership_check import get_tracks_positions_in_playlists
import json

# Load playlists from a JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)


# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [32]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-08.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  # Gets 'regional-au-daily-2024-02-05.csv'
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [33]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [34]:
# Assuming `track_ids` is your Pandas Series of track IDs as shown
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialize lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that you have all the data, you can put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame to verify the results
popularity_df.head()


,artist_name,track_name,popularity_score
0,Noah Kahan,Stick Season,97
1,Jack Harlow,Lovin On Me,99
2,Benson Boone,Beautiful Things,95
3,Tate McRae,greedy,100
4,Taylor Swift,Cruel Summer,99


In [35]:
df['popularity_score'] = popularity_df['popularity_score']

In [36]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [37]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,08-02-2024,1,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,126,339132,97
1,08-02-2024,2,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,3,91,284674,99
2,08-02-2024,3,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,2,21,268214,95
3,08-02-2024,4,Tate McRae,greedy,3rUGC1vUpkDG9CZFHMur1t,RCA Records Label,1,4,147,238015,100
4,08-02-2024,5,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,5,369,236102,99


In [38]:
df['previous_rank'].unique()

array([  1,   3,   2,   4,   5,   6,   7,   8,   9,  10,  12,  11,  16,
        15,  13,  17,  20,  14,  18,  19,  47,  21,  54,  22,  23,  25,
        24,  26,  31,  28,  33,  27,  29,  30,  32,  34,  37,  48,  36,
        40,  39,  35,  38,  45,  46,  41,  49,  43,  44,  50,  52,  51,
        53, 177,  42,  87,  65,  57,  58,  55,  63,  60,  62,  61,  59,
        56,  68,  73,  64,  67,  72,  71,  75,  66,  77,  79,  91,  74,
        -1,  83,  70,  78,  81,  69,  86, 106,  88,  76,  82,  92, 102,
        94,  84,  85,  93, 100,  95, 109,  89,  96, 104,  97,  80, 154,
        98, 103, 108, 101, 116,  90, 112, 155, 121,  99, 107, 118, 132,
       120, 126, 110, 140, 105, 117, 124, 113, 123, 114, 115, 111, 125,
       130, 148, 122, 127, 119, 138, 139, 196, 152, 129, 133, 134, 131,
       143, 161, 142, 135, 150, 141, 162, 137, 136, 175, 128, 146, 188,
       144, 170, 174, 157, 149, 160, 164, 185, 169, 153, 197, 167, 193,
       145, 182, 186, 158, 176, 147, 198, 166, 179, 181, 168, 19

Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [39]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [40]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

In [41]:
# Selecting specific columns to view
selected_columns_df = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]

# Display the DataFrame with the selected columns
selected_columns_df['rank_change'].unique()


array(['=', '+1', '-1', '+3', '-2', '-4', '+26', '+31', '-3', '+2', '-5',
       '+10', '-7', '+123', '-13', '+8', '-6', '-10', '+5', '-8', '+14',
       'RE', '-11', '-16', '+19', '-24', '+49', '+6', '-21', '+42', '+7',
       '-9', '+18', '-18', '-14', '-19', '+15', '-12', '-17', '+57',
       '+12', '+11', '-15', '+21', '-27', '+20', '+29', '+23', '+9',
       '-30', '-31', '-41', '-20', '-34', '-37'], dtype=object)

Need to change "=" to "0", remove "+", and handle "RE"

In [42]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "RE" at position 78
Non-numeric values found:
      rank rank_change                                     artist_title  \
78     79          RE                SZA - Open Arms (w/ Travis Scott)   
82     83          RE                             SZA - Nobody Gets Me   
170   171          RE                                        SZA - Low   
180   181          RE                  RAYE - Escapism. (w/ 070 Shake)   
182   183          RE  Fleetwood Mac - Go Your Own Way - 2004 Remaster   
187   188          RE     Daryl Hall & John Oates - You Make My Dreams   
192   193          RE       Florence + The Machine - Dog Days Are Over   
194   195          RE                               Hozier - Work Song   

     days_on_chart  peak_rank days_at_peak  daily_streams  \
78              13         65          NaN          69845   
82             108         12          NaN          68953   
170            154         26      

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_27620/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [43]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [44]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Noah Kahan - Stick Season,0,13.0,5318.0,2342179,-14010,22030359
1,Jack Harlow - Lovin On Me,1,72.0,8947.0,1991720,-10637,29063822
2,Benson Boone - Beautiful Things,-1,2.0,-16920.0,1819740,32681,3858887


In [45]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [46]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [47]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [48]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,08-02-2024,Noah Kahan,Stick Season,1,1,0,1,13.0,126,97,339132,5318.0,2342179,-14010,22030359,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,08-02-2024,Jack Harlow,Lovin On Me,2,3,1,1,72.0,91,99,284674,8947.0,1991720,-10637,29063822,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
2,08-02-2024,Benson Boone,Beautiful Things,3,2,-1,2,2.0,21,95,268214,-16920.0,1819740,32681,3858887,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
3,08-02-2024,Tate McRae,greedy,4,4,0,1,9.0,147,100,238015,-5816.0,1658226,291,33450170,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t
4,08-02-2024,Taylor Swift,Cruel Summer,5,5,0,1,1.0,369,99,236102,1769.0,1604879,13005,52126446,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          44 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  192 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [50]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,05-02-2024,Noah Kahan,Stick Season,1,1,0,1,10.0,123,97,333388,8284.0,2376830,-13316,21011273,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,05-02-2024,Jack Harlow,Lovin On Me,2,2,0,1,72.0,88,99,272882,1522.0,2070240,-45409,28217939,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
2,05-02-2024,Benson Boone,Beautiful Things,3,3,0,3,3.0,18,93,267772,28872.0,1611385,91911,3005801,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
3,05-02-2024,Tate McRae,greedy,4,4,0,1,9.0,144,100,232303,1031.0,1654249,-4706,32729268,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t
4,05-02-2024,Taylor Swift,Cruel Summer,5,5,0,1,1.0,366,99,220866,-2740.0,1560404,1221,51425313,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr


In [51]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [52]:
print(len(combined_df))

800


In [53]:
combined_df.to_csv('combined_data/combined.csv', index=False)

## Example usage of `get_tracks_position_in_playlist`

Example: Checking which playlists and positions Noah Kahan's 'Stick Season' is in

In [5]:
# Usage
track_ids = [
    '0mflMxspEfB0VbI1kyLiAv', # Noah Kahan 'Stick Season' track_id
    # ... add other track IDs
]

track_positions = get_tracks_positions_in_playlists(client_id, client_secret, redirect_uri, playlists_dict, track_ids)

for track_id, playlists in track_positions.items():
    print(f"Track ID {track_id} is in the following playlists at these positions:")
    for playlist_name, positions in playlists.items():
        positions_str = ", ".join(str(position + 1) for position in positions)  # Create a string of positions, accounting for 0 indexing
        print(f"Playlist: {playlist_name}: Position #{positions_str}")
    if not playlists:
        print("This track is not in any of the given playlists.")

Track ID 0mflMxspEfB0VbI1kyLiAv is in the following playlists at these positions:
Playlist: Hot Hits Australia: Position #5
